In [2]:
# create function
import pandas as pd
import numpy as np
import changefinder
import tabpy_client

/Users/ymasachika/.pyenv/versions/anaconda3-4.2.0/envs/tabpy/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# create client
client = tabpy_client.Client('http://localhost:9004/')

In [60]:
# create changefinder
def cf_func(data, r, order, smooth):
    import changefinder
    cf = changefinder.ChangeFinder(r=r[0], order=order[0], smooth=smooth[0])
    ret = [cf.update(i) for i in data]
    return ret

client.deploy('cf_func',
              cf_func,
              'Returns change finder function',
              override=True)

In [71]:
# create kmenas function
def kmeans_func(n_clus, rstat, *args):
    import pandas as pd
    import numpy as np
    import numpy as np
    from sklearn.cluster import KMeans
    
    df = pd.DataFrame()
    for arg in args:
        df = pd.concat([df, pd.DataFrame(arg)] , axis=1)
    X = df.as_matrix()
    kmeans_model = KMeans(n_clusters=n_clus[0], random_state=rstat[0])
    db = kmeans_model.fit(X)
    return db.labels_.tolist()

client.deploy('kmeans',
              kmeans_func,
              'Returns kmeans label',
              override=True)

In [14]:
def randomforest_func(n_estimators, label, test_size, *args):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import numpy as np
    
    # 引数をDataFrameにする
    df = pd.DataFrame()
    for arg in args:
        df = pd.concat([df, pd.DataFrame(arg)] , axis=1)
    X = df.as_matrix()
    
    # Split Train and Test data
    X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=test_size[0], random_state=71)
    
    clf = RandomForestClassifier(n_estimators=n_estimators[0])
    clf.fit(X_train,y_train)
    yhat_train = clf.predict(X_train)
    print("Train Accuracy: {0}".format(sum(yhat_train == y_train) / len(y_train)))
    
    yhat_test = clf.predict(X_test)
    print("Test Accuracy: {0}".format(sum(yhat_test == y_test) / len(y_test)))
    
    result = clf.predict(X)
    return result.tolist()

client.deploy('randomforest',
              randomforest_func,
              'Returns predict label',
              override=True)